### Title: **Tiny Thesis** - Top view does NOT matter & should NOT be a part of our experiments moving forward

**Date:** 2026-02-18

**System:** Conda : act_train

**Goal:** Test 


### 1. Motivation

**Goal** : Show that ACT performance is impacted on Plug-insert when we remove certain camera views.

## 2. Task design and Hypothesis

### Task Description

Plug insertion is a multi-mode dexterous task. The policy must:
1. Upon seeing different tilt orientations, produce corrective motions.
2. Produce appropriate downward force to complete the insertion.

**Goal**: Show Plug-Insert performance is **not** degraded by removing the Top Camera. Benefits are:
1. Significantly reducing Grid Search domain.
2. Provide insights for producing "cripped" ACT baselines to test Force-Torque injection.

### Task Configurations
We perform the below tasks:
* TRAIN: 3 views on Plug insert - 30 eps
* TRAIN: 2 views on Plug insert - 30 eps
* TRAIN: 3 views on Plug insert - 60 eps
* TRAIN: 2 views on Plug insert - 60 eps



## 3. Task Design Considerations
Q: Why choose 30 / 60?\
A: These correspond to the dataset content in the **Plug Insert** Dataset.\
30 Episodes only contains the powerboard at a fixed location. The 60 Episode version contains positional variations in the dataset.


## 4. Method

NOTE: jupyter notebooks cannot prevent process kill even with nohup. We run below in a tmux terminal.

Below is the exact bash command I ran for scheduling the remaining Training.

In short, this is done by `while ps -p <PID> > /dev/null; do sleep 60; done; ./schedule_train.sh`.

The command is saved as a bash script, `schedule_train.sh`

TIP: Remember to **source the environment** in the Tmux session!!

## 5. Results + Evaluation
We can only run real world rollouts for each of the tests.\
For each training configuration, we run 20 rollouts to evaluate performance.

Results :

Failure modes:
* At certain more extreme angles, it cannot apply enough force to complete the insertion.
* A

### Policy 2

python3 imitate_episodes.py \
  --task_name plug_insert \
  --ckpt_dir ./ckpt_dir/plug_insert_base_no_left \
  --robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
  --policy_class ACT \
  --kl_weight 10 \
  --chunk_size 100 \
  --hidden_dim 512 \
  --batch_size 16 \
  --dim_feedforward 3200 \
  --num_epochs 6000 \
  --lr 1e-5 --seed 0 \
  --exclude_cameras camera_left_shoulder \
  --eval \
  --temporal_agg

### Results:

Actually this policy has severe difficulty performing the insertion.\
This suggests the top view cam IS important to get successful feedback.

### Policy 3

python3 imitate_episodes.py \
  --task_name plug_insert \
  --ckpt_dir ./ckpt_dir/plug_insert_base60 \
  --robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
  --policy_class ACT \
  --kl_weight 10 \
  --chunk_size 100 \
  --hidden_dim 512 \
  --batch_size 16 \
  --dim_feedforward 3200 \
  --num_epochs 8000 \
  --lr 1e-5 --seed 0 \
  --eval \
  --temporal_agg

**Results:**
* Pretty good generalization when within distribution
* However, during rollout it tends to stop in its tracks, but is able to fix it during the next rollout.
* Running it 3 times in a row, it can make enough microadjustments to pretty much fix any misalignment.
* At further away, slightly out of distribution poses, this is not as obvious.

### Policy 4

python3 imitate_episodes.py \
  --task_name plug_insert \
  --ckpt_dir ./ckpt_dir/plug_insert_base60_no_left \
  --robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
  --policy_class ACT \
  --kl_weight 10 \
  --chunk_size 100 \
  --hidden_dim 512 \
  --batch_size 16 \
  --dim_feedforward 3200 \
  --num_epochs 8000 \
  --lr 1e-5 --seed 0 \
  --exclude_cameras camera_left_shoulder  \
  --eval \
  --temporal_agg

**Results:**
* Slightly weaker than policy 3, but not as obviously as between Policy 1 and 2.

## Design considerations 
* When policy gets stuck, it should be able to reset back to a wider view position, dynamically recompute the temporal aggregation / temporal chunk size to allow reset.
* Compliance controller might give it this tendency.

## Open Questions / Next Steps